In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/WhatApp Chat/New chats/zip files/*.zip' -d '/content/drive/MyDrive/WhatApp Chat/New chats/unzip files'

Archive:  /content/drive/MyDrive/WhatApp Chat/New chats/zip files/WhatsApp Chat with +880 1670-253960.zip
  inflating: /content/drive/MyDrive/WhatApp Chat/New chats/unzip files/WhatsApp Chat with +880 1670-253960.txt  
  inflating: /content/drive/MyDrive/WhatApp Chat/New chats/unzip files/IMG-20230505-WA0003.jpg  
  inflating: /content/drive/MyDrive/WhatApp Chat/New chats/unzip files/IMG-20240314-WA0002.jpg  

Archive:  /content/drive/MyDrive/WhatApp Chat/New chats/zip files/WhatsApp Chat with +880 1987-474395.zip
  inflating: /content/drive/MyDrive/WhatApp Chat/New chats/unzip files/WhatsApp Chat with +880 1987-474395.txt  

Archive:  /content/drive/MyDrive/WhatApp Chat/New chats/zip files/WhatsApp Chat with +880 1911-584999.zip
  inflating: /content/drive/MyDrive/WhatApp Chat/New chats/unzip files/WhatsApp Chat with +880 1911-584999.txt  

Archive:  /content/drive/MyDrive/WhatApp Chat/New chats/zip files/WhatsApp Chat with +880 1713-620367.zip
  inflating: /content/drive/MyDrive/What

In [ ]:
import re
import os
import csv

In [ ]:
def parse_chat(file_path, filename):
  with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

    #customer_text = []
    #support_text = []
    conversation = []

    filename = filename.replace('WhatsApp Chat with', '')

    for line in lines:
      # Extract customer and support messages

      if filename in line:
        line = line.replace(filename, 'Customer')

      # else:
      #   line = line.replace('Lahore by IKitchen', 'Support')

      conversation.append(line.strip())

      # if 'Lahore by IKitchen' in line:
      #   support_text.append(line.strip())
      # else:
      #   customer_text.append(line.strip())

    entry = {
        'Name/Phone Number': filename,
        'Conversation': '\n'.join(conversation)
        # 'Customer Text': '\n'.join(customer_text),
        # 'Support Text': '\n'.join(support_text)
    }

    return entry

In [ ]:
def process_directory(directory):
  data = []

  for filename in os.listdir(directory):
    if filename.endswith('.txt'):
      file_path = os.path.join(directory, filename)
      #Use filename without extension as the unique ID
      unique_id = os.path.splitext(filename) [0]
      entry = parse_chat(file_path, unique_id)
      data.append(entry)

  return data

In [ ]:
def save_to_csv(data, output_file):
  with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Name/Phone Number', 'Conversation']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for entry in data:
      writer.writerow(entry)


In [ ]:
directory = '/content/drive/MyDrive/WhatApp Chat/New chats/unzip files'
output_file = '400_whatsapp_chat.csv'
data = process_directory(directory)
save_to_csv(data, output_file)

print('Conversion complete. Data saved to', output_file)

Conversion complete. Data saved to 400_whatsapp_chat.csv


In [ ]:
import pandas as pd
import re

# Read the CSV file
df = pd.read_csv('/content/400_whatsapp_chat.csv')

# Function to remove date and time patterns
def remove_date_time(text):
    # Remove patterns like "27/06/2024, 3:44 am - "
    text = re.sub(r'\d{2}/\d{2}/\d{4},\s+\d{1,2}:\d{2}\s+[ap]m\s+-\s*', '', text)
    # Remove phone numbers
    text = re.sub(r'\+\d{3,}\s+\d+-\d+:\s*', '', text)
    text = text.replace('Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.', '')

    return text.strip()

# Apply the cleaning function to 'Customer Text' and 'Support Text' columns
df['Conversation'] = df['Conversation'].apply(remove_date_time)
#df['Support Text'] = df['Support Text'].apply(remove_date_time)

# Save the cleaned data to a new CSV file
df.to_csv('/content/drive/MyDrive/WhatApp Chat/New chats/cleaned_data.csv', index=False)

print("Data cleaning completed. Check 'cleaned_data.csv' for the result.")

Data cleaning completed. Check 'cleaned_data.csv' for the result.
